# Initialize spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.streaming import StreamingContext

spark = SparkSession \
    .builder \
    .appName("CloudProject") \
    .getOrCreate()


In [ ]:
from pyspark.sql.types import *
scc = StreamingContext(spark,4)

# Initialize data streaming

In [ ]:
MONITOR_SOURCE='/home/guglielmo/universita/cloud_computing/progetto/code/mock-data/monitor'
P_SENSORS_SOURCE='/home/guglielmo/universita/cloud_computing/progetto/code/mock-data/measurements/pollution'
T_SENSORS_SOURCE='/home/guglielmo/universita/cloud_computing/progetto/code/mock-data/measurements/temperature'

### Initialize monitor data streaming

In [ ]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("device_health", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("area", StringType(), True),
    StructField("customer", StringType(), True),
])
monitor_stream = spark.readStream.schema(schema).format("json").option('path',MONITOR_SOURCE).load()
monitor_stream

In [ ]:
monitor_stream.isStreaming

### Initialize sensors data streaming

In [ ]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("humidity", IntegerType(), True),
    StructField("temperature", IntegerType(), True)
])
t_sensors_stream = spark.readStream.schema(schema).format("json").option('path',T_SENSORS_SOURCE).load()
t_sensors_stream

In [ ]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("CO2_level", IntegerType(), True)
])
p_sensors_stream = spark.readStream.schema(schema).format("json").option('path',P_SENSORS_SOURCE).load()
p_sensors_stream

In [ ]:
print(monitor_stream.isStreaming, t_sensors_stream.isStreaming, p_sensors_stream.isStreaming)

In [ ]:
monitor_query = monitor_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

t_sensors_query = t_sensors_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

p_sensors_query = p_sensors_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

In [ ]:
monitor_query.stop()
t_sensors_query.stop()
p_sensors_query.stop()